In [154]:
import matplotlib.pyplot as plt
import pickle
import numpy as np
import torch
from tqdm import tqdm
from collections import defaultdict, Counter

In [162]:
frame_set = []
with open('output_summary_without_feedback/P03_UTE_policy_grad_summary_length_600_subshot_size_200_hidden_dim_256_summary_without_feedback.txt','r') as f:
    frame_set = f.readlines()
frame_set = [1 if i=='1\n' else 0 for i in frame_set]
Counter(frame_set)

Counter({1: 600, 0: 9484})

In [168]:
gt_summaries = []
gt_data = ['gr01_1.csv','gr01_2.csv','gr01_3.csv']
for fl in gt_data:
    with open(f'datasets/videoset_data/reference_summaries/gr/{fl}','r') as f:
        data = f.readlines()
        data = data[1:]
        data_nums = [int(i[-11:-5]) for i in data[:-1]]
        data_nums.append(int(data[-1][-10:-4]))
    gt_summary = [0]*len(frame_set)
    for i in data_nums:
        for num in range(i*5,i*5+5):
            num = min(len(frame_set)-1,num)
            gt_summary[num]=1
    gt_summaries.append(gt_summary)

In [169]:
frame_set = []
with open('output_summary_without_feedback/P03_UTE_policy_grad_summary_length_600_subshot_size_200_hidden_dim_256_summary_without_feedback.txt','r') as f:
    frame_set = f.readlines()
frame_set = [1 if i=='1\n' else 0 for i in frame_set]
Counter(frame_set)

Counter({1: 600, 0: 9484})

In [170]:
gt_summaries = []
gt_data = ['gr03_1.csv','gr03_2.csv','gr03_3.csv']
for fl in gt_data:
    with open(f'datasets/videoset_data/reference_summaries/gr/{fl}','r') as f:
        data = f.readlines()
        data = data[1:]
        data_nums = [int(i[-11:-5]) for i in data[:-1]]
        data_nums.append(int(data[-1][-10:-4]))
    gt_summary = [0]*len(frame_set)
    for i in data_nums:
        for num in range(i*5,i*5+5):
            num = min(len(frame_set)-1,num)
            gt_summary[num]=1
    gt_summaries.append(gt_summary)

In [171]:
len(frame_set)//5

2016

In [172]:
Counter(gt_summaries[0]), Counter(gt_summaries[1])

(Counter({0: 9798, 1: 286}), Counter({0: 9838, 1: 246}))

In [173]:
frame_set = np.array(frame_set)
gt_summary = np.array(gt_summary)

In [174]:
def relaxed_f1_score(predicted_summary, ground_truth_summary, delta_t):
    """
    Calculate the relaxed F1 score based on a fixed temporal relaxation (∆t).

    Args:
    - predicted_summary (list): Binary sequence representing the predicted summary.
    - ground_truth_summary (list): Binary sequence representing the ground truth summary.
    - delta_t (int): Temporal relaxation threshold.

    Returns:
    - f1_score (float): Relaxed F1 score.
    """
    # Convert binary sequences to list of intervals
    def binary_to_intervals(binary_sequence):
        intervals = []
        start = None
        for i, value in enumerate(binary_sequence):
            if value == 1 and start is None:
                start = i
            elif value == 0 and start is not None:
                intervals.append((start, i - 1))
                start = None
        if start is not None:
            intervals.append((start, len(binary_sequence) - 1))
        return intervals

    # Calculate the intersection over union (IoU) of two intervals
    def calculate_iou(interval1, interval2):
        intersection = max(0, min(interval1[1], interval2[1]) - max(interval1[0], interval2[0]) + delta_t+1)
        union = max(interval1[1] - interval1[0] + delta_t+1, interval2[1] - interval2[0] + delta_t+1) - intersection
        return intersection / union if union > 0 else 0

    # Convert binary sequences to intervals
    pred_intervals = binary_to_intervals(predicted_summary)
    gt_intervals = binary_to_intervals(ground_truth_summary)

    tp, fp, fn = 0, 0, 0

    for pred_interval in pred_intervals:
        iou_with_ground_truth = max([calculate_iou(pred_interval, gt_interval) for gt_interval in gt_intervals])

        if iou_with_ground_truth > 0:
            tp += 1
        else:
            fp += 1

    for gt_interval in gt_intervals:
        iou_with_prediction = max([calculate_iou(gt_interval, pred_interval) for pred_interval in pred_intervals])

        if iou_with_prediction == 0:
            fn += 1

    precision = tp / (tp + fp) if tp + fp > 0 else 0
    recall = tp / (tp + fn) if tp + fn > 0 else 0

    f1_score = 2 * (precision * recall) / (precision + recall) if precision + recall > 0 else 0

    return f1_score


delta_t = 50
avg_f1 = 0.0
for gt_summary in gt_summaries:
    relaxed_f1 = relaxed_f1_score(frame_set, gt_summary, delta_t)
    print("Relaxed F1 Score:", relaxed_f1)
    avg_f1+=relaxed_f1
print(avg_f1/len(gt_summaries))


Relaxed F1 Score: 0.5676190476190476
Relaxed F1 Score: 0.6773049645390072
Relaxed F1 Score: 0.6365280289330922
0.6271506803637156
